# 6. Hand shape image classification

## 6.2 initial learning

In [2]:
import os
import sys
import glob
import numpy as np
from skimage import io
from sklearn import datasets

In [3]:
#define constant
IMAGE_SIZE = 40
COLOR_BYTE = 3
CATEGORY_NUM = 6

In [13]:
#function for getting hand image

def load_handimage(path):
    files = glob.glob(os.path.join(path, '*/*.png'))
    
    #allocate for image and label
    images = np.ndarray((len(files), IMAGE_SIZE,IMAGE_SIZE, COLOR_BYTE),
                       dtype = np.uint8)
    labels = np.ndarray(len(files), dtype=np.int)
    
    for idx, file in enumerate(files):
        #load image
        image = io.imread(file)
        images[idx] = image
        
        #get label name (※dir name represents label)
        label = os.path.split(os.path.dirname(file))[-1]
        labels[idx] = label
    
    #follow other datasets' format in scikit-learn
    print(images.shape)
    flat_data = images.reshape((-1, IMAGE_SIZE*IMAGE_SIZE*COLOR_BYTE))
    print(flat_data.shape)
    images = flat_data.view()
    return datasets.base.Bunch(data=flat_data,
                              target=labels.astype(np.int),
                              target_names=np.arange(CATEGORY_NUM),
                              images=images,
                              DESCR=None)
